In [3]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [4]:
import pandas as pd
import io

# read csv
train_df = pd.read_csv(io.StringIO(uploaded['train.csv'].decode('utf-8')))

# safety: limit number of rows to display
pd.set_option("display.max_rows", 10)

# look at data
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [9]:
# find the min, max, median, and mean fo the SalePrice
print (train_df.SalePrice.min())
print (train_df.SalePrice.max())
print (train_df.SalePrice.median())
print (train_df.SalePrice.mean())

34900
755000
163000.0
180921.19589041095


In [10]:
# find the different types of Street and Alley
print (train_df.Street.unique())
print (train_df.Alley.unique())

['Pave' 'Grvl']
[nan 'Grvl' 'Pave']


In [11]:
# find the MSSubClass that appears most often
train_df.MSSubClass.value_counts()

20     536
60     299
50     144
120     87
30      69
      ... 
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, Length: 15, dtype: int64

In [13]:
# re-map the SalPrice by subtracting the mean
mp = train_df.SalePrice.mean()
train_df.SalePrice.map(lambda sp: sp - mp)

0       27578.80411
1         578.80411
2       42578.80411
3      -40921.19589
4       69078.80411
           ...     
1455    -5921.19589
1456    29078.80411
1457    85578.80411
1458   -38796.19589
1459   -33421.19589
Name: SalePrice, Length: 1460, dtype: float64

In [16]:
# find the house with the most LotArea to SalePrice ratio
train_df.loc[(train_df.LotArea / train_df.SalePrice).idxmax()]

Id                  336
MSSubClass          190
MSZoning             RL
LotFrontage         NaN
LotArea          164660
                  ...  
MoSold                8
YrSold             2008
SaleType             WD
SaleCondition    Normal
SalePrice        228950
Name: 335, Length: 81, dtype: object

In [26]:
# find the most commmon combination of LotShape and MSZoning
ans = train_df.loc[(train_df.LotShape.notnull()) & (train_df.MSZoning.notnull())]
ans = ans.apply(lambda srs: srs.LotShape + " - " + srs.MSZoning, axis='columns')
ans.value_counts()

Reg - RL         652
IR1 - RL         455
Reg - RM         199
Reg - FV          52
IR2 - RL          35
                ... 
IR2 - RM           2
IR1 - RH           2
IR1 - C (all)      1
IR2 - RH           1
IR3 - RM           1
Length: 16, dtype: int64

In [27]:
# find how man times each type of MSZoning occurs
train_df.groupby('MSZoning').MSZoning.count()

MSZoning
C (all)      10
FV           65
RH           16
RL         1151
RM          218
Name: MSZoning, dtype: int64

In [5]:
# find the biggest LotArea for a given SalePrice
train_df.groupby('LotArea').SalePrice.max().sort_index()

LotArea
1300      124000
1477       80000
1491       75500
1526       86000
1533       97000
           ...  
70761     280000
115149    302000
159000    277000
164660    228950
215245    375000
Name: SalePrice, Length: 1073, dtype: int64

In [6]:
# find the max and min sale price of houses grouped by MSZoning
train_df.groupby('MSZoning').SalePrice.agg([min, max])

,min,max
MSZoning,,
C (all),34900,133900
FV,144152,370878
RH,76000,200000
RL,39300,755000
RM,37900,475000


In [11]:
# find the max and min sale price of houses grouped by MSZoning
train_df.groupby('MSZoning').SalePrice.mean()

MSZoning
C (all)     74528.000000
FV         214014.061538
RH         131558.375000
RL         191004.994787
RM         126316.830275
Name: SalePrice, dtype: float64

In [12]:
# find the max and min LotFrontage of the various MSSubClass types
train_df.groupby('MSSubClass').LotFrontage.agg([min, max])

,min,max
MSSubClass,,
20,36.0,313.0
30,30.0,153.0
40,40.0,60.0
45,50.0,60.0
50,40.0,152.0
...,...,...
90,35.0,110.0
120,30.0,68.0
160,21.0,75.0


In [14]:
# find the most common combinationof LotShape and LandContour
train_df['n'] = 0
train_df.groupby(['LotShape', 'LandContour']).n.count().sort_values(ascending=False)

LotShape  LandContour
Reg       Lvl            861
IR1       Lvl            416
Reg       Bnk             39
IR1       HLS             32
IR2       Lvl             29
                        ... 
          HLS              4
          Bnk              3
IR3       Low              2
          Bnk              2
          HLS              1
Name: n, Length: 16, dtype: int64

In [0]:
#minimal_privacy = train_df.Fence.map(lambda r: r.Fence == 'MnPrv').value_counts()
#good_privacy = train_df.Fence.map(lambda r: r.Fence == 'GdPrv').value_counts()
#pd.Series([minimal_privacy[True], good_privacy[True]], index=['Min Privacey', 'Good Privacy'])